In [ ]:
# THIS NOTEBOOK CLEANS AND PREPS THE TESTING DATASET FOR USE IN ML MODELS
# IT A COPY OF THE TRAIN_DATA_PREP NOTEBOOK BUT READS IN THE TESTING CSV

In [1]:
# Import the required modules
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression

In [5]:
# Read in Testing csv
fraud_test_data = pd.read_csv(
    Path('fraud_test_dataset.csv'))
fraud_test_data.head()

,Unnamed: 0.1,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,1,1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2,2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,3,3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,4,4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [6]:
# Drop unnamed columns
fraud_test_data = fraud_test_data.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])
fraud_test_data.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,Titusville,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,Falmouth,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


In [7]:
# Check target variable imbalance
fraud_test_data['is_fraud'].value_counts()

is_fraud
0    99598
1      402
Name: count, dtype: int64

In [8]:
# Check dtypes
fraud_test_data.dtypes

trans_date_trans_time     object
cc_num                     int64
merchant                  object
category                  object
amt                      float64
first                     object
last                      object
gender                    object
street                    object
city                      object
state                     object
zip                        int64
lat                      float64
long                     float64
city_pop                   int64
job                       object
dob                       object
trans_num                 object
unix_time                  int64
merch_lat                float64
merch_long               float64
is_fraud                   int64
dtype: object

In [9]:
# Drop unnecessary columns
fraud_test_data = fraud_test_data.drop(columns=['cc_num', 'first', 'last', 'trans_num'])

In [12]:
# Assign categorical columns to new variable to be encoded
categorical_data = fraud_test_data[['trans_date_trans_time', 'merchant', 'category', 'gender', 
                               'street', 'city', 'state', 'job', 'dob']].copy()
categorical_data.head()

,trans_date_trans_time,merchant,category,gender,street,city,state,job,dob
0,2020-06-21 12:14:25,fraud_Kirlin and Sons,personal_care,M,351 Darlene Green,Columbia,SC,Mechanical engineer,1968-03-19
1,2020-06-21 12:14:33,fraud_Sporer-Keebler,personal_care,F,3638 Marsh Union,Altonah,UT,"Sales professional, IT",1990-01-17
2,2020-06-21 12:14:53,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,F,9333 Valentine Point,Bellmore,NY,"Librarian, public",1970-10-21
3,2020-06-21 12:15:15,fraud_Haley Group,misc_pos,M,32941 Krystal Mill Apt. 552,Titusville,FL,Set designer,1987-07-25
4,2020-06-21 12:15:17,fraud_Johnston-Casper,travel,M,5783 Evan Roads Apt. 465,Falmouth,MI,Furniture designer,1955-07-06


In [13]:
# Create a loop that will iterate through and encode the categorical variables
for column in categorical_data.columns:
    frequency_map = categorical_data[column].value_counts().to_dict()
    categorical_data[column + 'freq_enc'] = categorical_data[column].map(frequency_map)

categorical_data.head()

,trans_date_trans_time,merchant,category,gender,street,city,state,job,dob,trans_date_trans_timefreq_enc,merchantfreq_enc,categoryfreq_enc,genderfreq_enc,streetfreq_enc,cityfreq_enc,statefreq_enc,jobfreq_enc,dobfreq_enc
0,2020-06-21 12:14:25,fraud_Kirlin and Sons,personal_care,M,351 Darlene Green,Columbia,SC,Mechanical engineer,1968-03-19,1,151,7046,45217,120,120,2291,451,120
1,2020-06-21 12:14:33,fraud_Sporer-Keebler,personal_care,F,3638 Marsh Union,Altonah,UT,"Sales professional, IT",1990-01-17,1,130,7046,54783,142,142,839,437,142
2,2020-06-21 12:14:53,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,F,9333 Valentine Point,Bellmore,NY,"Librarian, public",1970-10-21,1,164,6639,54783,193,193,6456,448,193
3,2020-06-21 12:15:15,fraud_Haley Group,misc_pos,M,32941 Krystal Mill Apt. 552,Titusville,FL,Set designer,1987-07-25,1,133,6260,45217,131,131,3319,131,131
4,2020-06-21 12:15:17,fraud_Johnston-Casper,travel,M,5783 Evan Roads Apt. 465,Falmouth,MI,Furniture designer,1955-07-06,1,73,3139,45217,167,167,3552,167,167


In [14]:
categorical_data.dtypes

trans_date_trans_time            object
merchant                         object
category                         object
gender                           object
street                           object
city                             object
state                            object
job                              object
dob                              object
trans_date_trans_timefreq_enc     int64
merchantfreq_enc                  int64
categoryfreq_enc                  int64
genderfreq_enc                    int64
streetfreq_enc                    int64
cityfreq_enc                      int64
statefreq_enc                     int64
jobfreq_enc                       int64
dobfreq_enc                       int64
dtype: object

In [15]:
# Drop original, unencoded variables so only encoded versions remain
categorical_data = categorical_data.drop(columns=['trans_date_trans_time', 'merchant', 'category', 'gender', 
                               'street', 'city', 'state', 'job', 'dob'])
categorical_data.head()

,trans_date_trans_timefreq_enc,merchantfreq_enc,categoryfreq_enc,genderfreq_enc,streetfreq_enc,cityfreq_enc,statefreq_enc,jobfreq_enc,dobfreq_enc
0,1,151,7046,45217,120,120,2291,451,120
1,1,130,7046,54783,142,142,839,437,142
2,1,164,6639,54783,193,193,6456,448,193
3,1,133,6260,45217,131,131,3319,131,131
4,1,73,3139,45217,167,167,3552,167,167


In [18]:
# Assign floats to new variable to next concatenate encoded categorical data with floats
floats = fraud_test_data.drop(['trans_date_trans_time', 'merchant', 'category', 'gender', 'street', 'city', 'state', 'job', 'dob'], axis=1)
floats.head()

,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
0,2.86,29209,33.9659,-80.9355,333497,1371816865,33.986391,-81.200714,0
1,29.84,84002,40.3207,-110.4360,302,1371816873,39.450498,-109.960431,0
2,41.28,11710,40.6729,-73.5365,34496,1371816893,40.495810,-74.196111,0
3,60.05,32780,28.5697,-80.8191,54767,1371816915,28.812398,-80.883061,0
4,3.19,49632,44.2529,-85.0170,1126,1371816917,44.959148,-85.884734,0


In [19]:
# Concat categorical data and floats back together
test_data = pd.concat([categorical_data, floats], axis=1)
test_data.head()

,trans_date_trans_timefreq_enc,merchantfreq_enc,categoryfreq_enc,genderfreq_enc,streetfreq_enc,cityfreq_enc,statefreq_enc,jobfreq_enc,dobfreq_enc,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
0,1,151,7046,45217,120,120,2291,451,120,2.86,29209,33.9659,-80.9355,333497,1371816865,33.986391,-81.200714,0
1,1,130,7046,54783,142,142,839,437,142,29.84,84002,40.3207,-110.4360,302,1371816873,39.450498,-109.960431,0
2,1,164,6639,54783,193,193,6456,448,193,41.28,11710,40.6729,-73.5365,34496,1371816893,40.495810,-74.196111,0
3,1,133,6260,45217,131,131,3319,131,131,60.05,32780,28.5697,-80.8191,54767,1371816915,28.812398,-80.883061,0
4,1,73,3139,45217,167,167,3552,167,167,3.19,49632,44.2529,-85.0170,1126,1371816917,44.959148,-85.884734,0


In [20]:
# Confirm dtypes are model friendly
test_data.dtypes

trans_date_trans_timefreq_enc      int64
merchantfreq_enc                   int64
categoryfreq_enc                   int64
genderfreq_enc                     int64
streetfreq_enc                     int64
cityfreq_enc                       int64
statefreq_enc                      int64
jobfreq_enc                        int64
dobfreq_enc                        int64
amt                              float64
zip                                int64
lat                              float64
long                             float64
city_pop                           int64
unix_time                          int64
merch_lat                        float64
merch_long                       float64
is_fraud                           int64
dtype: object

In [21]:
# Export as csv to be loaded into Random Forest Notebook
test_data.to_csv('prepped_fraud_test_dataset.csv')